In [3]:
import pandas as pd

df = pd.read_csv("valid_reports.csv", sep=",")
df["VolumeName"]

0     /root/data_volumes/dataset/valid/valid_1/valid...
1     /root/data_volumes/dataset/valid/valid_2/valid...
2     /root/data_volumes/dataset/valid/valid_3/valid...
3     /root/data_volumes/dataset/valid/valid_4/valid...
4     /root/data_volumes/dataset/valid/valid_4/valid...
5     /root/data_volumes/dataset/valid/valid_5/valid...
6     /root/data_volumes/dataset/valid/valid_6/valid...
7     /root/data_volumes/dataset/valid/valid_7/valid...
8     /root/data_volumes/dataset/valid/valid_7/valid...
9     /root/data_volumes/dataset/valid/valid_8/valid...
10    /root/data_volumes/dataset/valid/valid_9/valid...
11    /root/data_volumes/dataset/valid/valid_10/vali...
12    /root/data_volumes/dataset/valid/valid_11/vali...
13    /root/data_volumes/dataset/valid/valid_12/vali...
14    /root/data_volumes/dataset/valid/valid_13/vali...
15    /root/data_volumes/dataset/valid/valid_14/vali...
16    /root/data_volumes/dataset/valid/valid_15/vali...
17    /root/data_volumes/dataset/valid/valid_16/